In [1]:
import os
import shutil
import datetime
import arcpy

tbx = r"C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\toolbox\tenement-tools-toolbox.pyt"
arcpy.ImportToolbox(tbx)

<module 'toolbox'>

In [2]:
# need button to create shapefile with relevent attributes
# set a folder
# create a shapefile of polygons
# need a id field, out netcdf, platform, start/end monitor date, other

In [4]:
out_folder = r'C:\Users\Lewis\Desktop\nrt_projects'
out_filename = 'ophthalmia_monitoring5'

# create a new nrt project
arcpy.NRT_Create_Project_toolbox(out_folder, out_filename)

Creating new monitoring project database...
Generating database feature class...
Generating database domains...
Generating database fields...
Generating database defaults...
Creating NetCDF data folder...
Adding data to current map...
Created new monitoring project database successfully.
monitoring_areas


<Result ''>